# 2i2c hub staffing model

How many people do we need to run x number of hubs?

The goal is to get the kind of functions right - we are really just guessing the co-efficients. 

Currently we think the following things scale sub-linearly:

1. Initial setup of the hub
2. Service improvements to the hubs

The following things scale linearly

1. Ongoing communication with the folks using the hubs
2. Incident response when things go bad

We then guess the co-efficients to make the graphs 'look ok'. So trust the numbers along the axes with big bags of salt - the shape is what we're trying to get right.

A *low touch* hub is deployed via [this setup](https://github.com/2i2c-org/similar-hubs), ideally with a GUI. Everything else is high-touch.

In [139]:
import altair as alt
from altair import datum
import pandas as pd
from math import e

In [140]:
def staffing_model(n):
    factors = {
        'low_touch': {
            'initial_setup': {
                'junior': lambda n: 0.4 * (e ** float(-n)),
                'senior': lambda n: 0.1 * (e ** float(-n))
            },
            'service_improvements': {
                'junior': lambda n: 0.3 * (e ** float(-n)),
                'senior': lambda n: 0.3 * (e ** float(-n))
            },
            'ongoing_communication': {
                'junior': lambda n: 0.05 * n,
                'senior': lambda n: 0.001 * n
            },
            'incident_response': {
                'junior': lambda n: 0.01 * n,
                'senior': lambda n: 0.01 * n
            }
        },
        'high_touch': {
            'initial_setup': {
                'junior': lambda n: 0.3 * (e ** float(-n)),
                'senior': lambda n: 0.1 * (e ** float(-n))
            },
            'service_improvements': {
                'junior': lambda n: 0.5 * (e ** float(-n)),
                'senior': lambda n: 0.4 * (e ** float(-n))
            },
            'ongoing_communication': {
                'junior': lambda n: 0.05 * n,
                'senior': lambda n: 0.01 * n
            },
            'incident_response': {
                'junior': lambda n: 0.05 * n,
                'senior': lambda n: 0.05 * n
            }
        }
        
    }
    data = []
    for i in range(1, n):
        entry = {
            'junior': 0,
            'senior': 0
        }
        for engineer in entry:
            for hub_type in factors:
                for factor in factors[hub_type]:
                    entry[engineer] += factors[hub_type][factor][engineer](i)
                data.append({
                    'hubs': i,
                    'engineer_type': engineer,
                    'hub_type': hub_type,
                    'count': entry[engineer]
                })
           
    return pd.DataFrame(data)

In [141]:
data = staffing_model(100)

alt.Chart(data[data['hub_type']=='low_touch']).mark_point().encode(x='hubs', y='count', color='engineer_type')

alt.Chart(...)

In [142]:
alt.Chart(data[data['hub_type']=='high_touch']).mark_point().encode(x='hubs', y='count', color='engineer_type')

alt.Chart(...)